In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
from PIL import Image, ImageFilter

In [2]:
# load data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
print('load data')

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
load data


In [3]:
mnist.test.images[0].shape

(784,)

In [ ]:
x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

#1layer
w_conv1 = tf.Variable(tf.truncated_normal([5,5,1,32],stddev=0.1))

conv1 = tf.nn.relu(tf.nn.conv2d(x_image, w_conv1, strides=[1, 1, 1, 1], padding='SAME'))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))

relu_conv1 = tf.nn.relu(conv1 + b_conv1)
pool1 = tf.nn.max_pool(relu_conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')     #output size 14


#2layer
w_conv2 = tf.Variable(tf.truncated_normal([5,5,32,64],stddev=0.1))

conv2 = tf.nn.relu(tf.nn.conv2d(pool1, w_conv2, strides=[1, 2, 2, 1], padding='SAME'))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))

relu_conv2 = tf.nn.relu(conv2 + b_conv2)
pool2 = tf.nn.max_pool(relu_conv1, ksize=[1,2,2,1], strides=[1,4,4,1], padding='SAME')    #output size 7


#fully connected layer
pool2_flat = tf.reshape(pool2, [-1, 7*7*32])   #1차원 행렬로 변환

w3 = tf.Variable(tf.truncated_normal([7*7*32, 1024]))
b3 = tf.Variable(tf.constant(0.1, shape=[1024]))

relu3 = tf.nn.relu(tf.matmul(pool2_flat, w3) + b3)

keep_prob = tf.placeholder(tf.float32)
drop3 = tf.nn.dropout(relu3, keep_prob)



w0 = tf.Variable(tf.zeros([1024, 10]))    #0~9를 찾는거니까 제일 마지막은 10
b0 = tf.Variable(tf.zeros([10]))
k = tf.matmul(drop3, w0) + b0

p = tf.nn.softmax(k)
y = tf.placeholder(tf.float32, [None, 10])

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(k,y)) 
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(50)
    sess.run(train_step, feed_dict={x: batch_xs, 
                                    y: batch_ys, 
                                    keep_prob: 0.5})
#     print(batch_ys[0].shape)

    

In [18]:
correct_prediction = tf.equal(tf.argmax(p,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels,
                                   keep_prob: 1}))

0.951
